# Цели исследования

Необходимо проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Провести исследование этих данных и получить результаты, которые помогут сформулировать ценностное предложение для нового продукта.

# Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Настройка доступа к базе данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine 

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Исследование таблиц

In [3]:
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for table in tables:
    query = '''
            SELECT * 
            FROM {}
            LIMIT 5;
        '''.format(table)
    display(pd.io.sql.read_sql(query, con=engine))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задача № 1
Посчитайте, сколько книг вышло после 1 января 2000 года

In [4]:
query = '''
            SELECT COUNT(*)
            FROM books
            WHERE publication_date > '2000-01-01';
        '''

pd.io.sql.read_sql(query, con=engine)

,count
0,819


### Вывод:
После 1 января 2000 года вышло 819 книг.

## Задача № 2
Для каждой книги посчитайте количество обзоров и среднюю оценку

In [5]:
query = '''
            WITH
            avg_rating_table AS (
                SELECT
                    b.book_id,
                    ROUND(AVG(rating), 2) AS avg_rating
                FROM books b 
                JOIN ratings r ON b.book_id = r.book_id
                GROUP BY b.book_id
            ),
            reviews_count_table AS (
                SELECT 
                    b.book_id,
                    COUNT(review_id) AS reviews_count
                FROM books b 
                JOIN reviews rw ON b.book_id = rw.book_id
                GROUP BY b.book_id 
            )
            
            SELECT 
                title,
                avg_rating,
                reviews_count
            FROM books b 
            LEFT JOIN avg_rating_table art ON b.book_id = art.book_id
            LEFT JOIN reviews_count_table rct ON b.book_id = rct.book_id;
        '''

pd.io.sql.read_sql(query, con=engine)

,title,avg_rating,reviews_count
0,The Body in the Library (Miss Marple #3),4.50,2.0
1,Galápagos,4.50,2.0
2,A Tree Grows in Brooklyn,4.25,5.0
3,Undaunted Courage: The Pioneering First Missio...,4.00,2.0
4,The Prophet,4.29,4.0
...,...,...,...
995,The Cat in the Hat and Other Dr. Seuss Favorites,5.00,NaN
996,Anne Rice's The Vampire Lestat: A Graphic Novel,3.67,NaN
997,Essential Tales and Poems,4.00,NaN
998,Leonardo's Notebooks,4.00,NaN


### Вывод:
Для каждой книги посчитано количество обзоров и средняя оценка.

## Задача № 3
Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [6]:
query = '''
            SELECT 
                publisher,
                COUNT(book_id) AS books_count
            FROM books b
            JOIN publishers p ON b.publisher_id = p.publisher_id
            WHERE num_pages > 50
            GROUP BY publisher
            ORDER BY books_count DESC
            LIMIT 1;
        '''

pd.io.sql.read_sql(query, con=engine)

,publisher,books_count
0,Penguin Books,42


### Вывод:

Penguin Books -- издательство, которое выпустило наибольшее число книг толще 50 страниц, а именно 42 книги.

## Задача № 4
Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [13]:
query = '''
            WITH
            books_table AS (
                SELECT 
                    b.book_id,
                    COUNT(rating) AS ratings_count
                FROM books b 
                JOIN ratings r ON b.book_id = r.book_id
                GROUP BY b.book_id
                HAVING COUNT(rating) >= 50
            )
            
            SELECT 
                author,
                ROUND(AVG(rating), 2) AS avg_rating
            FROM books_table bt 
            JOIN ratings r ON bt.book_id = r.book_id
            JOIN books b ON bt.book_id = b.book_id
            JOIN authors a ON b.author_id = a.author_id
            GROUP BY author
            ORDER BY avg_rating DESC
            LIMIT 1;
        '''

pd.io.sql.read_sql(query, con=engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


### Вывод:

J.K. Rowling/Mary GrandPré -- авторы с самой высокой средней оценкой книг, равной 4.29 .

## Задача № 5
Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [7]:
query = ''' 
            WITH
            users_table AS (
                SELECT username
                FROM ratings
                GROUP BY username
                HAVING COUNT(rating) > 50
            ),
            reviews_count_table AS (
                SELECT 
                    u.username,
                    COUNT(review_id) AS reviews_count
                FROM users_table u
                JOIN reviews r ON u.username=r.username
                GROUP BY u.username
            )
            
            SELECT ROUND(AVG(reviews_count)) AS avg_reviews_count
            FROM reviews_count_table;
        '''


pd.io.sql.read_sql(query, con=engine)

,avg_reviews_count
0,24.0


### Вывод:

24 -- среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Общий вывод

#### Что сделано:
- Осуществлена настройка доступа к базе данных
- Проведено исследование таблиц
- Решено 5 задач

#### Выводы и результаты:
- После 1 января 2000 года вышло 819 книг.
- Для каждой книги посчитано количество обзоров и средняя оценка.
- Penguin Books -- издательство, которое выпустило наибольшее число книг толще 50 страниц, а именно 42 книги.
- J.K. Rowling/Mary GrandPré -- авторы с самой высокой средней оценкой книг, равной 4.29 .
- 24 -- среднее количество обзоров от пользователей, которые поставили больше 50 оценок.